In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import sklearn
import torch

FILE_NAME_SUFFIX = ".multi_admission" #"_1000"
DATASET_PATH = "Datasets/"

def read_samples(input_file, file_name_suffix):
    file_name = DATASET_PATH + input_file + file_name_suffix
    return pd.read_csv(file_name, error_bad_lines=False, keep_default_na=False)


In [2]:
patients = read_samples("PATIENTS.csv", FILE_NAME_SUFFIX)
admissions = read_samples("ADMISSIONS.csv", FILE_NAME_SUFFIX)
admissions["admit_date"] = admissions["ADMITTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
admissions["discharge_date"] = admissions["DISCHTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
admissions['LOS'] = admissions["discharge_date"] - admissions["admit_date"]

diagnoses = read_samples("DIAGNOSES_ICD.csv", FILE_NAME_SUFFIX)
procedures = read_samples("PROCEDURES_ICD.csv", FILE_NAME_SUFFIX)
icu_stays = read_samples("ICUSTAYS.csv", FILE_NAME_SUFFIX)
notes = read_samples("NOTEEVENTS.csv", FILE_NAME_SUFFIX)


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
patients = patients.set_index("SUBJECT_ID", drop=False)
patients["num_admissions"] = admissions.groupby("SUBJECT_ID").size().to_frame("num_admissions")
patients = patients[patients.num_admissions > 1]
print("Limit the patients to the ones with more than 1 admission:\n", patients.num_admissions.describe())
print("Prevalence of patients with more than 1 admission: ", len(patients[patients["num_admissions"] > 1]) / len(patients))
print("Prevalence of patients with more than 2 admissions: ", len(patients[patients["num_admissions"] > 2]) / len(patients))
print("Prevalence of patients with more than 3 admissions: ", len(patients[patients["num_admissions"] > 3]) / len(patients))
print("Prevalence of patients with more than 4 admissions: ", len(patients[patients["num_admissions"] > 4]) / len(patients))
print("Prevalence of patients with more than 5 admissions: ", len(patients[patients["num_admissions"] > 5]) / len(patients))

Limit the patients to the ones with more than 1 admission:
 count    7537.000000
mean        2.652647
std         1.621112
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max        42.000000
Name: num_admissions, dtype: float64
Prevalence of patients with more than 1 admission:  1.0
Prevalence of patients with more than 2 admissions:  0.31537747114236436
Prevalence of patients with more than 3 admissions:  0.13732254212551412
Prevalence of patients with more than 4 admissions:  0.06992171951704923
Prevalence of patients with more than 5 admissions:  0.03728273849011543


In [4]:
admissions = admissions[admissions.SUBJECT_ID.isin(patients.SUBJECT_ID)]
procedures = procedures[procedures.SUBJECT_ID.isin(patients.SUBJECT_ID)]
diagnoses = diagnoses[diagnoses.SUBJECT_ID.isin(patients.SUBJECT_ID)]
icu_stays = icu_stays[icu_stays.SUBJECT_ID.isin(patients.SUBJECT_ID)]
notes = notes[notes.SUBJECT_ID.isin(patients.SUBJECT_ID)]

In [5]:
last_admission = admissions[admissions.groupby(['SUBJECT_ID'])['admit_date'].transform(max) == admissions['admit_date']]
previous_admissions = admissions[admissions.groupby(['SUBJECT_ID'])['admit_date'].transform(max) != admissions['admit_date']]
patients["record_start_date"] = previous_admissions.groupby("SUBJECT_ID").admit_date.agg(['min'])
patients["record_end_date"] = previous_admissions.groupby("SUBJECT_ID").discharge_date.agg(['max'])
patients["record_length"] = patients.record_end_date - patients.record_start_date
patients["final_admission_date"] = admissions.groupby("SUBJECT_ID").admit_date.agg(['max'])
patients["final_admission_interval"] = patients.final_admission_date - patients.record_end_date

print("Record length before the final admission\n", patients.record_length.describe())

print("Interval before the final admission\n", patients.final_admission_interval.describe())

print("Prevalence of readmission in 30 days = ", len(patients[patients.final_admission_interval < 30]) / len(patients))


Record length before the final admission
 count    7533.000000
mean      242.149608
std       579.288149
min         0.000000
25%         6.000000
50%        13.000000
75%        84.000000
max      4145.000000
Name: record_length, dtype: float64
Interval before the final admission
 count    7533.000000
mean      445.034382
std       678.907514
min       -19.000000
25%        25.000000
50%       131.000000
75%       570.000000
max      4108.000000
Name: final_admission_interval, dtype: float64
Prevalence of readmission in 30 days =  0.27517579938967757


In [6]:
icu_stays["admit_date"] = icu_stays["INTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
final_icu_admission = icu_stays[icu_stays.groupby(['SUBJECT_ID'])['admit_date'].transform(max) != icu_stays['admit_date']]
patients["final_icu_admission_date"] = final_icu_admission.groupby("SUBJECT_ID").admit_date.agg(['min'])
patients["final_icu_admission_interval"] = patients.final_icu_admission_date - patients.record_end_date
print("Prevalence of ICU admission in 30 days = ", len(patients[(patients["final_icu_admission_interval"] >= 0) & (patients["final_icu_admission_interval"] < 30)]) / len(patients))


Prevalence of ICU admission in 30 days =  0.006103224094467295


In [7]:
print(procedures.groupby("ICD9_CODE").size().to_frame("procedure_freq_by_icd9_code").describe())
print(procedures.groupby("SUBJECT_ID").size().to_frame("procedure_freq_by_patient").describe())
print(procedures.groupby("HADM_ID").size().to_frame("procedure_freq_by_admission").describe())

print(procedures.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(20))

       procedure_freq_by_icd9_code
count                  1513.000000
mean                     54.035030
std                     275.114179
min                       1.000000
25%                       1.000000
50%                       4.000000
75%                      19.000000
max                    6505.000000
       procedure_freq_by_patient
count                7364.000000
mean                   11.101983
std                     8.891169
min                     1.000000
25%                     5.000000
50%                     9.000000
75%                    15.000000
max                    98.000000
       procedure_freq_by_admission
count                 17393.000000
mean                      4.700454
std                       3.961372
min                       1.000000
25%                       2.000000
50%                       3.000000
75%                       6.000000
max                      40.000000
           freq
ICD9_CODE      
3893       6505
9604       3440
966      

In [8]:
print(diagnoses.groupby("ICD9_CODE").size().to_frame("diagnosis_freq_by_icd9_code").describe())
print(diagnoses.groupby("SUBJECT_ID").size().to_frame("diagnosis_freq_by_patient").describe())
print(diagnoses.groupby("HADM_ID").size().to_frame("diagnosis_freq_by_admission").describe())

print(diagnoses.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(20))

       diagnosis_freq_by_icd9_code
count                  4894.000000
mean                     53.192889
std                     258.042880
min                       1.000000
25%                       1.000000
50%                       4.000000
75%                      20.000000
max                    7183.000000
       diagnosis_freq_by_patient
count                7537.000000
mean                   34.539737
std                    28.731059
min                     2.000000
25%                    18.000000
50%                    28.000000
75%                    41.000000
max                   540.000000
       diagnosis_freq_by_admission
count                 19993.000000
mean                     13.020857
std                       6.860812
min                       1.000000
25%                       9.000000
50%                      11.000000
75%                      17.000000
max                      39.000000
           freq
ICD9_CODE      
4019       7183
4280       6588
42731    

In [9]:
notes['text_len'] = notes['TEXT'].apply(lambda s: len(s.split()))
print(notes["text_len"].describe())
print(notes.groupby("SUBJECT_ID").size().to_frame("notes_freq_by_patient").describe())
print(notes.groupby("HADM_ID").size().to_frame("notes_freq_by_admission").describe())

count    739127.000000
mean        280.506953
std         381.865602
min           0.000000
25%          72.000000
50%         160.000000
75%         318.000000
max        7980.000000
Name: text_len, dtype: float64
       notes_freq_by_patient
count            7535.000000
mean               98.092502
std               117.930373
min                 1.000000
25%                34.000000
50%                62.000000
75%               116.000000
max              1420.000000
       notes_freq_by_admission
count             19758.000000
mean                 37.408999
std                 948.520727
min                   1.000000
25%                   8.000000
50%                  15.000000
75%                  31.000000
max              133139.000000


In [10]:
discharge_summaries = notes[notes.CATEGORY == "Discharge summary"]
print(discharge_summaries["text_len"].describe())
print(discharge_summaries.groupby("SUBJECT_ID").size().to_frame("notes_freq_by_patient").describe())
print(discharge_summaries.groupby("HADM_ID").size().to_frame("notes_freq_by_admission").describe())

count    21740.000000
mean      1603.497148
std        884.371393
min          9.000000
25%       1008.000000
50%       1527.000000
75%       2111.000000
max       7980.000000
Name: text_len, dtype: float64
       notes_freq_by_patient
count            7451.000000
mean                2.917729
std                 1.935627
min                 1.000000
25%                 2.000000
50%                 2.000000
75%                 3.000000
max                47.000000
       notes_freq_by_admission
count             19050.000000
mean                  1.141207
std                   0.438223
min                   1.000000
25%                   1.000000
50%                   1.000000
75%                   1.000000
max                   7.000000


In [11]:
patients["death_date"] = patients["DOD"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days if s != '' else np.nan)
patients['death_interval'] = patients.death_date - patients.record_end_date
print(patients['death_interval'].describe())
print("Prevalence of death in 30 days = ", len(patients[patients.death_interval < 30]) / len(patients))
print("Prevalence of death = ", len(patients[patients.death_interval >= 0]) / len(patients))

count    3902.000000
mean      663.113275
std       811.485606
min         0.000000
25%        80.250000
50%       316.500000
75%       937.750000
max      4328.000000
Name: death_interval, dtype: float64
Prevalence of death in 30 days =  0.05532705320419265
Prevalence of death =  0.5177126177524214


In [12]:
# Limit the procedures to the most common procedures
NUM_PROCEDURE_CODES = 16
top_procedures = procedures.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(NUM_PROCEDURE_CODES).index.tolist()
procedures = procedures[procedures.ICD9_CODE.isin(top_procedures)]
print(procedures.groupby("ICD9_CODE").size().to_frame("procedure_freq_by_icd9_code").describe())
print(procedures.groupby("SUBJECT_ID").size().to_frame("procedure_freq_by_patient").describe())
print(procedures.groupby("HADM_ID").size().to_frame("procedure_freq_by_admission").describe())

       procedure_freq_by_icd9_code
count                    16.000000
mean                   2182.312500
std                    1401.472546
min                     982.000000
25%                    1333.250000
50%                    1558.500000
75%                    2832.000000
max                    6505.000000
       procedure_freq_by_patient
count                6459.000000
mean                    5.405945
std                     4.799150
min                     1.000000
25%                     2.000000
50%                     4.000000
75%                     7.000000
max                    50.000000
       procedure_freq_by_admission
count                 13165.000000
mean                      2.652260
std                       1.987886
min                       1.000000
25%                       1.000000
50%                       2.000000
75%                       3.000000
max                      27.000000


In [13]:
# Limit the diagnoses to the most common diagnoses
NUM_DIAGNOSIS_CODES = 16
top_diagnoses = diagnoses.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(NUM_DIAGNOSIS_CODES).index.tolist()
diagnoses = diagnoses[diagnoses.ICD9_CODE.isin(top_diagnoses)]
print(diagnoses.groupby("ICD9_CODE").size().to_frame("diagnosis_freq_by_icd9_code").describe())
print(diagnoses.groupby("SUBJECT_ID").size().to_frame("diagnosis_freq_by_patient").describe())
print(diagnoses.groupby("HADM_ID").size().to_frame("diagnosis_freq_by_admission").describe())

       diagnosis_freq_by_icd9_code
count                    16.000000
mean                   3490.125000
std                    1643.981625
min                    1915.000000
25%                    2195.750000
50%                    3021.000000
75%                    4216.750000
max                    7183.000000
       diagnosis_freq_by_patient
count                7008.000000
mean                    7.968322
std                     6.438043
min                     1.000000
25%                     4.000000
50%                     7.000000
75%                    10.000000
max                   102.000000
       diagnosis_freq_by_admission
count                 17466.000000
mean                      3.197183
std                       1.733602
min                       1.000000
25%                       2.000000
50%                       3.000000
75%                       4.000000
max                      12.000000


In [14]:
from torch.utils.data import Dataset

admission_encoding_size = len(top_procedures) + len(top_diagnoses) + 1
# encode each admission as
#   length of stay
#   frequency of each procedure in top_procedures
#   frequency of each diagnosis in top_diagnoses
#   TODO: add the embedding of the clinical notes here
                

class CustomDataset(Dataset):
    
    def __init__(self, patients, admissions, procedures, top_procedures, diagnoses, top_diagnoses, prediction_window):
        top_procedures_dict = dict(zip(top_procedures, range(len(top_procedures))))
        top_diagnoses_dict = dict(zip(top_diagnoses, range(len(top_diagnoses))))
        self.x = []
        self.y = []
        for _, patient in patients.iterrows():
            patient_admissions = []
            for _, admission in admissions[admissions.SUBJECT_ID == patient.SUBJECT_ID].iterrows():
                patient_admission = np.zeros(admission_encoding_size)
                patient_admission[0] = admission.LOS
                for _, admission_procedure in procedures[procedures.HADM_ID == admission.HADM_ID].iterrows():
                    patient_admission[1 + top_procedures_dict[admission_procedure.ICD9_CODE]] += 1
                for _, admission_diagnosis in diagnoses[diagnoses.HADM_ID == admission.HADM_ID].iterrows():
                    patient_admission[1 + len(top_procedures) + top_diagnoses_dict[admission_diagnosis.ICD9_CODE]] += 1
                patient_admissions.append(patient_admission)
            self.x.append(patient_admissions)
            # self.y.append([patient.final_admission_interval < prediction_window, patient.death_interval < prediction_window])
            self.y.append(patient.final_admission_interval < prediction_window)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

dataset = CustomDataset(patients, previous_admissions, procedures, top_procedures, diagnoses, top_diagnoses, 30)
print(len(dataset))
# for i in range(len(dataset)):
#     x, y = dataset[i]
#     print(y)

7537


In [15]:
def collate_fn(data):
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, encoding size). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, encoding size) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, encoding size) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (# patiens) of type torch.float
    """

    sequences, labels = zip(*data)
    num_patients = len(sequences)
    max_admissions = 0
    for sequence in sequences:
        max_admissions = max(max_admissions, len(sequence))
    
    dim = (num_patients, max_admissions, admission_encoding_size)
    x_data = np.zeros(dim)
    rev_x_data = np.zeros(dim)
    masks_data = np.full(dim, False)
    rev_masks_data = np.full(dim, False)
    
    for i, sequence in enumerate(sequences):
        num_admissions = len(sequence)
        for j, admission in enumerate(sequence):
            x_data[i][j] = admission
            rev_x_data[i][num_admissions-j-1] = admission
            masks_data[i][j] = np.ones_like(masks_data[i][j])
            rev_masks_data[i][num_admissions-j-1] = np.ones_like(rev_masks_data[i][j])

    x = torch.tensor(x_data, dtype=torch.float)
    rev_x = torch.tensor(rev_x_data, dtype=torch.float)
    masks = torch.tensor(masks_data, dtype=torch.bool)
    rev_masks = torch.tensor(rev_masks_data, dtype=torch.bool)
    y = torch.tensor(labels, dtype=torch.float)
    
    return x, masks, rev_x, rev_masks, y

In [16]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(loader)
x, masks, rev_x, rev_masks, y = next(loader_iter)
print(x.shape, masks.shape, rev_x.shape, rev_masks.shape, y.shape)

torch.Size([10, 5, 33]) torch.Size([10, 5, 33]) torch.Size([10, 5, 33]) torch.Size([10, 5, 33]) torch.Size([10])


In [17]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6029
Length of val dataset: 1508


In [18]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset, collate_fn):
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
    
    return train_loader, val_loader


train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)

In [19]:
def get_last_visit(hidden_states, masks):
    batch_size, visits, embedding_dim = hidden_states.shape
    masks = torch.sum(masks, 2)
    masks = torch.min(masks, torch.ones_like(masks))
    masks = torch.sum(masks, 1)
    masks = masks - torch.ones_like(masks)
    masks = masks.unsqueeze(1).expand(batch_size, embedding_dim).unsqueeze(1)
    masks = torch.max(masks, torch.zeros_like(masks)) # FIXME: data cleaning problem! some patients have no admission
    last_visit = torch.gather(hidden_states, 1, masks)
    last_visit = torch.flatten(last_visit, 1, 2)
    return last_visit

In [20]:
class NaiveRNN(torch.nn.Module):
    def __init__(self, admission_encoding_size, hidden_state_size, output_size):
        super().__init__()
        self.hidden_state_size = hidden_state_size
        self.rnn = torch.nn.GRU(admission_encoding_size, hidden_state_size, batch_first=True)
        self.linear1 = torch.nn.Linear(hidden_state_size, 32)
        self.linear2 = torch.nn.Linear(32, output_size)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.activation1 = torch.nn.ReLU()
        self.activation2 = torch.nn.Sigmoid()
    
    
    def forward(self, x, masks, rev_x, rev_masks):
        (batch_size, num_admissions, _) = x.shape
        hidden_state = torch.zeros(1, batch_size, self.hidden_state_size)
        hidden_states = []
        for admission in range(num_admissions):
            _, hidden_state = self.rnn(torch.narrow(x, 1, admission, 1), hidden_state)
            hidden_states.append(hidden_state)
        hidden_states = torch.cat(tuple(hidden_states), 0)
        hidden_states = torch.transpose(hidden_states, 0, 1)
        
        output = get_last_visit(hidden_states, masks)
        output = self.activation1(self.linear1(output))
        output = self.dropout(output)
        output = self.activation2(self.linear2(output))
        return output.squeeze()
    

# load the model here
naive_rnn = NaiveRNN(admission_encoding_size=admission_encoding_size, hidden_state_size=64, output_size=1)
naive_rnn

NaiveRNN(
  (rnn): GRU(33, 64, batch_first=True)
  (linear1): Linear(in_features=64, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (activation1): ReLU()
  (activation2): Sigmoid()
)

In [21]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)

In [22]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


def eval_model(model, val_loader):
    model.eval()
    Y1_pred = []
    Y1_true = []
    # Y2_pred = []
    # Y2_true = []
    for x, masks, rev_x, rev_masks, y in val_loader:
        pred = model(x, masks, rev_x, rev_masks)
        with torch.no_grad():
            pred = model(x, masks, rev_x, rev_masks)
            Y1_true.extend(y.detach().numpy().tolist())
            Y1_pred.extend(pred.detach().numpy().reshape(-1).tolist())
            # Y1_true.extend(y.detach().numpy()[:,0].tolist())
            # Y1_pred.extend(pred.detach().numpy()[:,0].reshape(-1).tolist())
            # Y2_true.extend(y.detach().numpy()[:,1].tolist())
            # Y2_pred.extend(pred.detach().numpy()[:,1].reshape(-1).tolist())
    
    precision1, recall1, f11, _ = precision_recall_fscore_support(Y1_true, np.array(Y1_pred)>0.5, average='binary')
    roc_auc1 = roc_auc_score(Y1_true, Y1_pred)
    # precision2, recall2, f12, _ = precision_recall_fscore_support(Y2_true, np.array(Y2_pred)>0.5, average='binary')
    # roc_auc2 = roc_auc_score(Y2_true, Y2_pred)
    
    return precision1, recall1, f11, roc_auc1# , precision2, recall2, f12, roc_auc2

In [25]:
precision1, recall1, f11, roc_auc1 = eval_model(naive_rnn, val_loader)
print('Task1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}'.format(precision1, recall1, f11, roc_auc1))

Task1: P=0.418 R=0.229 F1=0.296 ROC AUC=0.606


In [ ]:
def train(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_pred = model(x, masks, rev_x, rev_masks)
            loss = criterion(y_pred, y)
            # y_task1 = torch.narrow(y, 1, 0, 1)
            # y_pred_task1 = torch.narrow(y_pred, 1, 0, 1)
            # loss = criterion(y_pred_task1, y_task1)
            loss.backward()
            optimizer.step()
        # precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2 = eval_model(model, val_loader)
        # print('Epoch {}\n\tTask1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}\n\tTask2: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}'.format(epoch, precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2))
        precision1, recall1, f11, roc_auc1 = eval_model(model, val_loader)
        print('Epoch {}\n\tTask1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}\n'.format(epoch, precision1, recall1, f11, roc_auc1))
    
# number of epochs to train the model
n_epochs = 100
train(naive_rnn, train_loader, val_loader, n_epochs)

Epoch 0
	Task1: P=0.439 R=0.259 F1=0.326 ROC AUC=0.603

Epoch 1
	Task1: P=0.429 R=0.216 F1=0.288 ROC AUC=0.598

Epoch 2
	Task1: P=0.448 R=0.266 F1=0.334 ROC AUC=0.602

Epoch 3
	Task1: P=0.428 R=0.294 F1=0.348 ROC AUC=0.601

Epoch 4
	Task1: P=0.427 R=0.279 F1=0.337 ROC AUC=0.595

Epoch 5
	Task1: P=0.448 R=0.266 F1=0.334 ROC AUC=0.605

Epoch 6
	Task1: P=0.426 R=0.249 F1=0.314 ROC AUC=0.595

